# Distance-Based Time Series Classification

Distance-based classifiers compare whole series (or subsequences) with a distance metric such as
Euclidean distance or Dynamic Time Warping (DTW). kNN is the canonical example: classify a series
by the labels of its nearest neighbors under a chosen distance.

## Core idea
Given two sequences $x_{1:T}$ and $y_{1:S}$, DTW finds a warping path $\pi$ that aligns indices
to minimize cumulative cost. The DTW distance is

$$D_{DTW}(x, y) = \min_{\pi} \sum_{(i, j) \in \pi} (x_i - y_j)^2.$$

This flexibility makes DTW robust to phase shifts and local speed changes.

In [ ]:
import numpy as np
import plotly.graph_objects as go

np.random.seed(7)
t = np.linspace(0, 2 * np.pi, 40)
x = np.sin(t) + 0.1 * np.random.randn(len(t))
y = np.sin(t + 0.6) + 0.1 * np.random.randn(len(t))

def dtw_cost_path(a, b):
    n, m = len(a), len(b)
    cost = np.full((n + 1, m + 1), np.inf)
    cost[0, 0] = 0.0
    for i in range(1, n + 1):
        for j in range(1, m + 1):
            dist = (a[i - 1] - b[j - 1]) ** 2
            cost[i, j] = dist + min(cost[i - 1, j], cost[i, j - 1], cost[i - 1, j - 1])
    # backtrack best path
    i, j = n, m
    path = []
    while i > 0 and j > 0:
        path.append((i - 1, j - 1))
        steps = [(i - 1, j), (i, j - 1), (i - 1, j - 1)]
        costs = [cost[a, b] for a, b in steps]
        i, j = steps[int(np.argmin(costs))]
    path.reverse()
    return cost[1:, 1:], path

cost, path = dtw_cost_path(x, y)

fig = go.Figure(data=go.Heatmap(z=cost, colorscale='Blues'))
fig.add_trace(
    go.Scatter(
        x=[p[1] for p in path],
        y=[p[0] for p in path],
        mode='lines',
        line=dict(color='red', width=2),
        name='DTW path',
    )
)
fig.update_layout(title='DTW alignment path on cost matrix', xaxis_title='y index', yaxis_title='x index')
fig.show()


## Visual intuition
The red path shows which points are matched under DTW. A perfectly diagonal path means the
series are aligned in time. Deviations indicate local stretching or compression.

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(y=x, mode='lines+markers', name='Series x'))
fig.add_trace(go.Scatter(y=y, mode='lines+markers', name='Series y'))
fig.update_layout(title='Two sequences with a phase shift', xaxis_title='time index')
fig.show()


## sktime inventory for distance-based classifiers
Use the registry to list available estimators in your local sktime install. We filter by module
path to catch distance-based families.

In [ ]:
try:
    import pandas as pd
    from sktime.registry import all_estimators

    def _estimators_df():
        try:
            return all_estimators(estimator_types='classifier', as_dataframe=True)
        except TypeError:
            ests = all_estimators(estimator_types='classifier')
            return pd.DataFrame(
                [{'name': n, 'class': c.__name__, 'module': c.__module__} for n, c in ests]
            )

    df = _estimators_df()
    mask = df['module'].str.contains('distance', case=False, na=False)
    df_distance = df[mask].sort_values('name')
    df_distance
except Exception as exc:
    print('sktime not installed or registry unavailable:', exc)


## Minimal sktime example (optional)
If sktime is installed, you can run a quick kNN baseline with DTW.

In [ ]:
try:
    from sktime.datasets import load_basic_motions
    from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier
    from sklearn.metrics import classification_report

    X_train, y_train = load_basic_motions(split='train', return_X_y=True)
    X_test, y_test = load_basic_motions(split='test', return_X_y=True)

    clf = KNeighborsTimeSeriesClassifier(n_neighbors=1, distance='dtw')
    clf.fit(X_train, y_train)
    pred = clf.predict(X_test)
    print(classification_report(y_test, pred))
except Exception as exc:
    print('Install sktime to run the classifier demo:', exc)


## When to use
- Strong baseline for small to medium datasets where elastic alignment matters.
- DTW is expensive; consider windowing or approximation for large datasets.
- Distance-based models are sensitive to scaling, so normalize per series.